In [1]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [3]:
data_sample = load_dataset("HazSylvia/Fitness_Unformatted")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/928 [00:00<?, ? examples/s]

In [4]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Human', 'Assistant'],
        num_rows: 928
    })
})

In [5]:
# Convert to a pandas dataframe
updated_data = [{'HUman': item['Human'], 'Assistant': item['Assistant']} for item in data_sample['train']]
df = pd.DataFrame(updated_data)

In [6]:
df.head(5)

# Just extract the Symptoms
df['Assistant'] = df['Assistant'].apply(lambda x: ', '.join(x.split(', ')))
display(df.head())

,HUman,Assistant
0,can you recommend effective ab exercises,planks bicycle crunches and leg raises are gre...
1,what are some effective strategies for managin...,make time for relaxation take time to catch y...
2,how can i incorporate regular movement and phy...,take the stairs instead of the elevator whene...
3,are there any specific strategies for maintain...,eat a healthy diet eating a healthy diet rich...
4,how can i manage stress and maintain a healthy...,manage your time wisely prioritize your tasks...


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


In [8]:
# If you have an NVIDIA GPU attached, use 'cuda'
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    # If Apple Silicon, set to 'mps' - otherwise 'cpu' (not advised)
    try:
        device = torch.device('mps')
    except Exception:
        device = torch.device('cpu')

device

device(type='cuda')

In [9]:
# The tokenizer turns texts to numbers (and vice-versa)
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# The transformer
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
model

# Model params
BATCH_SIZE = 8

df.describe()

,HUman,Assistant
count,928,928
unique,720,719
top,how can i create a supportive and positive soc...,the cable seated row is a popular exercise to ...
freq,6,4


In [29]:
class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)  # Fix here
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):  # Fix here
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        return max_length

In [30]:
data_sample = LanguageDataset(df, tokenizer)

data_sample

In [31]:
# Create train, valid
train_size = int(0.8 * len(data_sample))
valid_size = len(data_sample) - train_size
train_data, valid_data = random_split(data_sample, [train_size, valid_size])


In [32]:
# Make the iterators
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)

In [33]:
num_epochs = 10

# Training parameters
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [34]:
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [35]:
# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])


In [36]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)


Training Epoch 1/10 Batch Size: 8, Transformer: distilgpt2:   0%|          | 0/93 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:

model.save_pretrained("/content/drive/MyDrive/Fine Tuning/fitnessmodel")
tokenizer.save_pretrained("/content/drive/MyDrive/Fine Tuning/fitnessmodel")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 50, 'do_sample': True}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


('/content/drive/MyDrive/Fine Tuning/fitnessmodel/tokenizer_config.json',
 '/content/drive/MyDrive/Fine Tuning/fitnessmodel/special_tokens_map.json',
 '/content/drive/MyDrive/Fine Tuning/fitnessmodel/vocab.json',
 '/content/drive/MyDrive/Fine Tuning/fitnessmodel/merges.txt',
 '/content/drive/MyDrive/Fine Tuning/fitnessmodel/added_tokens.json',
 '/content/drive/MyDrive/Fine Tuning/fitnessmodel/tokenizer.json')

In [39]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = "/content/drive/MyDrive/Fine Tuning/fitmodel"

# Load the tokenizer and model from the saved directory
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
result=generator("How can I get healthy hair?")
print(result)



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How can I get healthy hair? | to avoid excessively hot showers and avoid using harsh soaps and detergents | use natural oils to nourish your hair in moderation and avoid using harsh soaps and detergents\n\n use natural oils to nour'}]


In [22]:
result=generator("I am traveling right now,so can I maintain my fitness?")
print(result)

[{'generated_text': 'I am traveling right now,so can I maintain my fitness? what am i doing wrong | its normal to plan a brisk walk but its manageable pack resistance bands and light weights to support muscle and strength\n consider interval training choose interval training in your fitness'}]


In [23]:
result=generator("How can I improve my posture?")
print(result)

[{'generated_text': 'How can I improve my posture? | good posture is essential strengthen core muscles with exercises like planks and bridges be mindful of your sitting and standing positions consider yoga or pilates to enhance flexibility and alignment'}]


In [40]:
result=generator("what are some practical tips for staying hydrated throughout the day",max_length=600)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'what are some practical tips for staying hydrated throughout the day |  carry a reusable water bottle with you wherever you go and refill it throughout the day\n drink a glass of water as soon as you wake up in the morning even before you have coffee or tea\n add fresh fruits and herbs like citrus slices and mint leaves to your water for flavor\n replace sugary soda or other drinks with a glass of water\n during meals alternate between bites and sips of water \n during meals alternate between bites and sips of water \n during meals alternate between bites and sips of water \n during meals alternate between bites and sips of water \n during meals alternate between bites and sips of water \n'}]
